In [1]:
import numpy as np

In [2]:
np.set_printoptions(suppress=True)

## State

### 2D Kalman

For a 2D Kalman,  
**state vector:**  
Dimension d = 4  
$$ \mathbf{x} =
\begin{bmatrix}
x \\
y \\
\dot{x} \\
\dot{y}
\end{bmatrix}$$

**State Transition Matrix**  
Dimension is $d^2$ = 16  
$$F = 
\begin{bmatrix}
1 & 0 & dt & 0 \\
0 & 1 & 0 & dt \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1
\end{bmatrix}$$

## Kalman Filter design

Design a KF for the state provided above

In [22]:
x = np.array([0., 0., 0., 0.])[:,None]
P = 1000 * np.identity(4) # high uncertainty without correlation
u = np.array([0., 0., 0., 0.])[:,None] # assuming control vector dim. is same as dim. of state
F = np.array([[1., 0., 1., 0.],
              [0., 1., 0., 1.],
              [0., 0., 1., 0.],
              [0., 0., 0., 1.]]) #state transition matrix
H = np.array([[1., 0., 0., 0.],
              [0., 1., 0., 0.]]) # measurement fn. only measuring x and y (location)
R = np.array([[1., 0.],
              [0., 1.]]) # variance in measurement space
I = np.identity(4)
P


array([[1000.,    0.,    0.,    0.],
       [   0., 1000.,    0.,    0.],
       [   0.,    0., 1000.,    0.],
       [   0.,    0.,    0., 1000.]])

### Filter

In [23]:
measurements = [[1.,1.], [2., 2.], [3.,3.]]

In [24]:
def filter(x, P, debug=False):
    for n in range(len(measurements)):
        z = np.asarray(measurements[n])[:,None]
        

        # prediction update
        
        x = F @ x + u # predicted next time step state
        P = F @ P @ F.T # predicted uncertainty in next step

        if debug:
            print("Prediction")
            print(f"{x=}")
            print(f"{P=}")
            print("-"*60)
        
        # measurement update
        
        y = z - H @ x
        s = H @ P @ H.T + R
        K = P @ H.T @ np.linalg.inv(s)

        x = x + K @ y # measurement update in state
        P = (I - K @ H) @ P # measurement update in uncertainty
        
        if debug:
            print(f"Update On measuring {z=}")
            print(f"{x=}")
            print(f"{P=}")
            print("="*60)

    return x, P
        

In [26]:
filter(x, P)

(array([[2.99950091],
        [2.99950091],
        [0.99950125],
        [0.99950125]]),
 array([[0.83264071, 0.        , 0.49908584, 0.        ],
        [0.        , 0.83264071, 0.        , 0.49908584],
        [0.49908584, 0.        , 0.49875345, 0.        ],
        [0.        , 0.49908584, 0.        , 0.49875345]]))

### Other values - Example 1

In [27]:
x = np.array([4., 12., 0, 0.])[:,None]
P = 1000 * np.identity(4) # high uncertainty without correlation
P[:2, :2] = np.zeros(2) # removing uncertainty in x and y (keeping only vel. uncertainty)
u = np.array([0., 0., 0., 0.])[:,None] # assuming control vector dim. is same as dim. of state
dt = 0.1 # time step
F = np.array([[1., 0., dt, 0.],
              [0., 1., 0., dt],
              [0., 0., 1., 0.],
              [0., 0., 0., 1.]]) #state transition matrix
H = np.array([[1., 0., 0., 0.],
              [0., 1., 0., 0.]]) # measurement fn. only measuring x and y (location)
R = np.identity(2)*0.1 # variance in measurement space
I = np.identity(4)
measurements = [[i+5., 10.-2*i] for i in range(6)]
measurements, P, R

([[5.0, 10.0], [6.0, 8.0], [7.0, 6.0], [8.0, 4.0], [9.0, 2.0], [10.0, 0.0]],
 array([[   0.,    0.,    0.,    0.],
        [   0.,    0.,    0.,    0.],
        [   0.,    0., 1000.,    0.],
        [   0.,    0.,    0., 1000.]]),
 array([[0.1, 0. ],
        [0. , 0.1]]))

In [28]:
filter(x, P)

(array([[  9.99934073],
        [  0.00131854],
        [  9.99890122],
        [-19.99780244]]),
 array([[0.03955609, 0.        , 0.06592682, 0.        ],
        [0.        , 0.03955609, 0.        , 0.06592682],
        [0.06592682, 0.        , 0.10987804, 0.        ],
        [0.        , 0.06592682, 0.        , 0.10987804]]))

### Example 2

In [31]:
measurements = [[1+5*i, 4-4*i] for i in range(4)]
x = np.array([-4., 8., 0, 0])[:, None]
measurements, x

([[1, 4], [6, 0], [11, -4], [16, -8]],
 array([[-4.],
        [ 8.],
        [ 0.],
        [ 0.]]))

In [32]:
filter(x,P)

(array([[ 15.99333555],
        [ -7.99466844],
        [ 49.98333889],
        [-39.98667111]]),
 array([[0.05331556, 0.        , 0.1332889 , 0.        ],
        [0.        , 0.05331556, 0.        , 0.1332889 ],
        [0.1332889 , 0.        , 0.33322226, 0.        ],
        [0.        , 0.1332889 , 0.        , 0.33322226]]))

### Example 3

In [34]:
measurements = [[1, 17-2*i] for i in range(4)]
x = np.array([1., 19., 0, 0])[:, None]
measurements, x

([[1, 17], [1, 15], [1, 13], [1, 11]],
 array([[ 1.],
        [19.],
        [ 0.],
        [ 0.]]))

In [35]:
filter(x,P)

(array([[  1.        ],
        [ 11.00266578],
        [  0.        ],
        [-19.99333555]]),
 array([[0.05331556, 0.        , 0.1332889 , 0.        ],
        [0.        , 0.05331556, 0.        , 0.1332889 ],
        [0.1332889 , 0.        , 0.33322226, 0.        ],
        [0.        , 0.1332889 , 0.        , 0.33322226]]))